In [38]:
import pandas as pd 
import re
import requests
import pickle

In [39]:
data = requests.get('https://www.ams.usda.gov/mnreports/gx_gr110.txt').text
data
data2 = data.split('\r\n')
data2

['GX_GR110',
 'Springfield, IL    Wed, Apr 03, 2019    USDA-IL Dept of Ag Market News',
 '',
 'Chicago Terminal Grain Report',
 '',
 'To Arrive Truck and Rail Bids for Grain Delivered to Chicago.  The',
 'following quotations represent Bids ($/ bu) from Terminal Elevators,',
 'Processors, Mills, and Merchandisers after 1:30 p.m. today.',
 '',
 'Grain          Bids        Delivery        Change           Basis       Change',
 '',
 'SRW Wheat  4.5400-4.7100   30 Days          UP 7        -17K to OptK     UNCH',
 '',
 'Soybeans   8.5375-8.5875     Spot          DN 1.25      -45K to -40K     UNCH',
 'Soybeans   8.5375-8.5875  15-30 Days       DN 1.25      -45K to -40K     UNCH',
 '',
 'Terminal Elevator Bids',
 'Corn       3.2725-3.4525     Spot          UP 1.25      -35K to -17K     UNCH',
 'Corn       3.2725-3.4525  15-30 Days       UP 1.25      -35K to -17K     UNCH',
 '',
 'Processor Bids',
 'Corn       3.5225-3.6625     Spot          UP 1.25       -10K to 4K      UNCH',
 'Corn       3

In [40]:
split = ['Grain','Terminal Elevator Bids', 'Processor Bids', 'Changes']
crops = [ 'SRW Wheat', 'Soybeans', 'Corn']
cols = ['grain', 'bids', 'delivery', 'change1', 'basis', 'change2']

In [41]:
r = re.compile(r'|'.join(split))
reports = re.split(r,data) #splits based on crop
reps = [x.split('\r\n') for x in reports]
#reps = 
reps

[['GX_GR110',
  'Springfield, IL    Wed, Apr 03, 2019    USDA-IL Dept of Ag Market News',
  '',
  'Chicago Terminal '],
 [' Report', '', 'To Arrive Truck and Rail Bids for '],
 [' Delivered to Chicago.  The',
  'following quotations represent Bids ($/ bu) from Terminal Elevators,',
  'Processors, Mills, and Merchandisers after 1:30 p.m. today.',
  '',
  ''],
 ['          Bids        Delivery        Change           Basis       Change',
  '',
  'SRW Wheat  4.5400-4.7100   30 Days          UP 7        -17K to OptK     UNCH',
  '',
  'Soybeans   8.5375-8.5875     Spot          DN 1.25      -45K to -40K     UNCH',
  'Soybeans   8.5375-8.5875  15-30 Days       DN 1.25      -45K to -40K     UNCH',
  '',
  ''],
 ['',
  'Corn       3.2725-3.4525     Spot          UP 1.25      -35K to -17K     UNCH',
  'Corn       3.2725-3.4525  15-30 Days       UP 1.25      -35K to -17K     UNCH',
  '',
  ''],
 ['',
  'Corn       3.5225-3.6625     Spot          UP 1.25       -10K to 4K      UNCH',
  'Corn     

In [42]:
list1 = []
for x in reps[:-1]:
    new = [ y for y in x if (y.startswith('SRW Wheat') or y.startswith('Soybeans') or y.startswith('Corn')) ]
    if new !=[]:
        list1.append(new)
list1

[['SRW Wheat  4.5400-4.7100   30 Days          UP 7        -17K to OptK     UNCH',
  'Soybeans   8.5375-8.5875     Spot          DN 1.25      -45K to -40K     UNCH',
  'Soybeans   8.5375-8.5875  15-30 Days       DN 1.25      -45K to -40K     UNCH'],
 ['Corn       3.2725-3.4525     Spot          UP 1.25      -35K to -17K     UNCH',
  'Corn       3.2725-3.4525  15-30 Days       UP 1.25      -35K to -17K     UNCH'],
 ['Corn       3.5225-3.6625     Spot          UP 1.25       -10K to 4K      UNCH',
  'Corn       3.5325-3.6625  15-30 Days       UP 1.25        -9K to 4K      UNCH']]

In [43]:
split_type = []
for n,x in enumerate(list1):
    for y in x:
        split_type.append(split[n])
split_type # this give the right column for the data frame

['Grain',
 'Grain',
 'Grain',
 'Terminal Elevator Bids',
 'Terminal Elevator Bids',
 'Processor Bids',
 'Processor Bids']

In [44]:
list3= []
for x in list1:
    for y in x:
        list3.append(re.split(r'\s{2,}',y)) 
        
df3= pd.DataFrame(list3,columns=cols)
df3['location'] = split_type
df3.loc[df3['location']=='Grain','location'] = 'Merchandiser'
df3['date'] = pd.Timestamp.now()
df3

,grain,bids,delivery,change1,basis,change2,location,date
0,SRW Wheat,4.5400-4.7100,30 Days,UP 7,-17K to OptK,UNCH,Merchandiser,2019-04-04 13:51:07.674387
1,Soybeans,8.5375-8.5875,Spot,DN 1.25,-45K to -40K,UNCH,Merchandiser,2019-04-04 13:51:07.674387
2,Soybeans,8.5375-8.5875,15-30 Days,DN 1.25,-45K to -40K,UNCH,Merchandiser,2019-04-04 13:51:07.674387
3,Corn,3.2725-3.4525,Spot,UP 1.25,-35K to -17K,UNCH,Terminal Elevator Bids,2019-04-04 13:51:07.674387
4,Corn,3.2725-3.4525,15-30 Days,UP 1.25,-35K to -17K,UNCH,Terminal Elevator Bids,2019-04-04 13:51:07.674387
5,Corn,3.5225-3.6625,Spot,UP 1.25,-10K to 4K,UNCH,Processor Bids,2019-04-04 13:51:07.674387
6,Corn,3.5325-3.6625,15-30 Days,UP 1.25,-9K to 4K,UNCH,Processor Bids,2019-04-04 13:51:07.674387


In [45]:
df3['bids1'] = df3.bids.str.split('-',expand=True)[0]
df3['bids2'] = df3.bids.str.split('-', expand = True)[1]
df3['change1'] = df3['change1'].str.replace('\s+',' ')
#[A-Za-z]|
try:
    
    df3['change1_1'] = df3['change1'].str.split('-', expand=True)[0]
    df3['change1_2'] = df3['change1'].str.split('-', expand=True)[1]
    df3['change2_1'] = df3['change2'].str.split('-', expand=True)[0]
    df3['change2_2'] = df3['change2'].str.split('-', expand=True)[1]

except KeyError:
    
    df3['change1_1'] =  df3['change1']
    df3['change1_2'] = None
    df3['change2_1'] = df3['change2']
    df3['change2_2'] = None
     
    
try:  
    df3['basis1'] = df3['basis'].str.split('to', expand=True)[0].str.replace('[A-Za-z]',"")
    df3['basis2'] = df3['basis'].str.split('to', expand=True)[1].str.replace('[A-Za-z]',"")
    
except KeyError:
    df3['basis1'] = df3['basis']
    df3['basis2'] = None

df3['contract'] = df3['basis'].str.split('to', expand=True)[0].str.replace('\d|[+-]',"")
df3

,grain,bids,delivery,change1,basis,change2,location,date,bids1,bids2,change1_1,change1_2,change2_1,change2_2,basis1,basis2,contract
0,SRW Wheat,4.5400-4.7100,30 Days,UP 7,-17K to OptK,UNCH,Merchandiser,2019-04-04 13:51:07.674387,4.5400,4.7100,UP 7,None,UNCH,None,-17,,K
1,Soybeans,8.5375-8.5875,Spot,DN 1.25,-45K to -40K,UNCH,Merchandiser,2019-04-04 13:51:07.674387,8.5375,8.5875,DN 1.25,None,UNCH,None,-45,-40,K
2,Soybeans,8.5375-8.5875,15-30 Days,DN 1.25,-45K to -40K,UNCH,Merchandiser,2019-04-04 13:51:07.674387,8.5375,8.5875,DN 1.25,None,UNCH,None,-45,-40,K
3,Corn,3.2725-3.4525,Spot,UP 1.25,-35K to -17K,UNCH,Terminal Elevator Bids,2019-04-04 13:51:07.674387,3.2725,3.4525,UP 1.25,None,UNCH,None,-35,-17,K
4,Corn,3.2725-3.4525,15-30 Days,UP 1.25,-35K to -17K,UNCH,Terminal Elevator Bids,2019-04-04 13:51:07.674387,3.2725,3.4525,UP 1.25,None,UNCH,None,-35,-17,K
5,Corn,3.5225-3.6625,Spot,UP 1.25,-10K to 4K,UNCH,Processor Bids,2019-04-04 13:51:07.674387,3.5225,3.6625,UP 1.25,None,UNCH,None,-10,4,K
6,Corn,3.5325-3.6625,15-30 Days,UP 1.25,-9K to 4K,UNCH,Processor Bids,2019-04-04 13:51:07.674387,3.5325,3.6625,UP 1.25,None,UNCH,None,-9,4,K


In [46]:
final = df3[['grain','location','contract','bids1','bids2','basis1','basis2','change1_1', 'change1_2','change2_1','change2_2','date']]
final = final.fillna('NA')
final

,grain,location,contract,bids1,bids2,basis1,basis2,change1_1,change1_2,change2_1,change2_2,date
0,SRW Wheat,Merchandiser,K,4.5400,4.7100,-17,,UP 7,NA,UNCH,NA,2019-04-04 13:51:07.674387
1,Soybeans,Merchandiser,K,8.5375,8.5875,-45,-40,DN 1.25,NA,UNCH,NA,2019-04-04 13:51:07.674387
2,Soybeans,Merchandiser,K,8.5375,8.5875,-45,-40,DN 1.25,NA,UNCH,NA,2019-04-04 13:51:07.674387
3,Corn,Terminal Elevator Bids,K,3.2725,3.4525,-35,-17,UP 1.25,NA,UNCH,NA,2019-04-04 13:51:07.674387
4,Corn,Terminal Elevator Bids,K,3.2725,3.4525,-35,-17,UP 1.25,NA,UNCH,NA,2019-04-04 13:51:07.674387
5,Corn,Processor Bids,K,3.5225,3.6625,-10,4,UP 1.25,NA,UNCH,NA,2019-04-04 13:51:07.674387
6,Corn,Processor Bids,K,3.5325,3.6625,-9,4,UP 1.25,NA,UNCH,NA,2019-04-04 13:51:07.674387


In [47]:
pickle.dump(final,open('gxgr110_frame','wb'))

In [48]:
final.to_csv('gxgr110.csv')